In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab3.ipynb")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy.stats as stats
from resources.hashutils import *

---

<h1><center>SDSE Lab 3 <br><br> K-means clustering </center></h1>

---

# 1) Load the data
Load the dataset using the function `load_data()` function, call it `D`.  Save the number of samples (rows) as `N` and the number of features (columns) as `P`.

In [ ]:
def load_data():
    with open('resources/data2d.pickle','rb') as f:
        D = pickle.load(f)
    return D
    
...
N = ...    # TODO
P = ...    # TODO

In [ ]:
grader.check("q1")

# 2) Scatter plot
Create a scatter plot with the first column of `D` along the x-axis and the second column along the y-axis. Use `plt.axis('equal')` to force the two axes to have the same scale. This is important for correct visualization of the Euclidean distances between points. Your figure should look like the one below.

Food for thought: How many clusters do you see?

<img src="resources/2.png" />

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.subplot()
...  # TODO

# 3) Random placement of centroids
Write a function that takes an integer `K` and the data matrix `D` as inputs, and returns `K` centroids randomly placed within the box spanned by the columns of `D`. The shape of the return value should be `(K,P)`. 

The template also takes a third optional parameter. This is used by the autograder to ensure repeatable results. 

**Note**: You should use `stats.uniform.rvs(loc=..., scale=..., size=...)` to generate random centroid placements. 

In [ ]:
def initialize_centroids(K,D,random_seed=None):
    if random_seed is not None:
        np.random.seed(seed=random_seed)
    P = D.shape[1]
    C = np.empty((K,P))
    ...    # TODO
    return C

In [ ]:
grader.check("q3")

# 4) Compute the assignment for given centroids
Write a function that takes the data matrix `D` (shape `(N,P)`) and centroid locations `C` (shape `(K,P)`) as inputs, and returns an assignment matrix `gamma`. `gamma` should be a numpy array with shape `(N,K)`. Each row of `gamma` should contain `K-1` `False` values and one `True` value in the position corresponding to the centroid that is nearest to that data point (Euclidean distance). 

Hint: `np.argmin`.

In [ ]:
def assign_samples_to_nearest_centroid(D,C):
    N = ...  # TODO
    K = ...  # TODO
    gamma = np.zeros((N,K),dtype=bool)  # Initialized to all False
    ...  # TODO
    return gamma

In [ ]:
D = load_data()
C4 = initialize_centroids(4,D,random_seed=234)
gamma4 = assign_samples_to_nearest_centroid(D,C4)

In [ ]:
C8 = initialize_centroids(8,D,random_seed=324)
gamma8 = assign_samples_to_nearest_centroid(D,C8)

In [ ]:
grader.check("q4")

# 5) Plot data, centroids, assignment
Write a function that takes as input the data matrix `D`, a centroids matrix `C`, and an assignment matrix `gamma`, and creates a scatter plot similar to the one shown here. The centroids should be indicated with a star ('*') with the marker edge color set to black. The data samples should be color-coded according to their cluster membership. Your plot may not look identical to this one due to the random placement of the centroids.

<img src="resources/5.png" />


Hints:
+ The colors to use is given in the list called `colors`.
+ You can use the `plot` function with input arguments `markersize`, `markersize`, `markeredgecolor`, and `color`.

In [ ]:
colors = ['tab:blue','tab:orange','tab:green','tab:red']

def plot_data(D, C, gamma):
    K = ...  # TODO
    plt.figure(figsize=(5,5))
    ax = plt.subplot()
    for k in range(K):
        ax.plot(...,...,'.',color=...,markersize=...)  # data
        ax.plot(...,...,'*',color=...,markersize=...,markeredgecolor=...) # centroids
    ax.axis('equal')

In [ ]:
plot_data(D, C4, gamma4)

# 6) Compute the centroids for a given assignment
Write a function that takes the assignment `gamma` and the data matrix `D` and returns centroids placed at the mean of each cluster. The return value should be a numpy array with shape `(K,D)`.

In [ ]:
def place_centroids(gamma, D):
    K = gamma.shape[1]
    P = D.shape[1]
    C = np.empty((K,P))
    ...    # TODO
    return C

In [ ]:
D = load_data()
newC4 = place_centroids(gamma4, D)

In [ ]:
grader.check("q6")

# 7) Code K-means
Now we will implement the K-means algorithm using the methods that we've coded so far. Implement the K-means algorithm described in class. 

The function `run_kmeans` takes the dataset `D` and initial centroids `C` as input. It returns the final placement of the centroids `C`, and the assignment matrix `gamma`. There are two stopping conditions:
+ Stop and return `C=None` and `gamma=None` if any of the clusters becomes empty. 
+ Stop if the assignment matrix does not change from one iteration to the next. 

In [ ]:
def run_kmeans(D,C):

    done = False
    first = True
        
    while not done:
        
        # compute assignment (E step)
        gamma = ...
        
        # place centroids (M step)
        C = ...
        
        # Break if any of the clusters are empty
        # Return C=None and gamma=None
        if ...:
            ...
            
        # Break if there is no change in the assignment
        if ...:
            ...
            
        # update gamma
        gammaold = gamma
        first = False
            
    return C, gamma

In [ ]:
D = load_data()
optC4, optgamma4 = run_kmeans(D,C4)

In [ ]:
optC8, optgamma8 = run_kmeans(D,C8)

In [ ]:
grader.check("q7")

# 8) Evaluate the cost function 
Write a function that evaluates the cost function for a given dataset `D`, centroid placement `C`, and assignment matrix `gamma`. The cost function is the sum of the squared distances from each point (row in `D`) to its assigned centroid (row in `C`). The function should return `np.nan` if `C` is `None`.

In [ ]:
def eval_cost(D,C,gamma):
    if C is None:
        return np.nan
    K = ...   # TODO
    cost = ...   # TODO (may involve a loop)
    return cost

In [ ]:
J4 = eval_cost(D,optC4,optgamma4)

In [ ]:
J8 = eval_cost(D,optC8,optgamma8)

In [ ]:
grader.check("q8")

# 9) Ensemble run
Write a function that runs K-means multiple times, each with a different initial centroid placement. These initial placements should be chosen with `initialize_centroids(K,D)`. The function returns `best_run`, which is a dictionary containing the centroids (`C`), cost (`cost`), and assignment matrix (`gamma`) for the best of the ensemble of runs. 

The `ensemblesize` argument of the `ensemble_run` function is the number of runs of K-means to execute. 

Note that the function also takes `random_seed` as a fourth argument. This is used by the autograder to ensure repeatability of the results. You should **not** pass `random_seed` to `initialize_centroids`.

In [ ]:
def ensemble_run(K,D,ensemblesize,random_seed=None):
    if random_seed is not None:
        np.random.seed(seed=random_seed)
    best_run = {'C':None,'gamma':None,'cost':None}
    for e in range(ensemblesize):
        
        # Run Kmeans with K randomly initialized centroids
        # If an empty cluster is obtained, then repeat, and keep on repeating 
        # until the result has no empty clusters
        ...   # TODO

        # Evaluate the cost function 
        ...   # TODO

        # Check whether this is the best result obtained so far, 
        # and if so, store it in best_run.
        ...   # TODO
  
    return best_run

In [ ]:
D = load_data()
best_run_K4 = ensemble_run(4,D,ensemblesize=100,random_seed=5423)

In [ ]:
grader.check("q9")

# 10) Iteration over K
Complete the function below that takes the data matrix `D`, an ensemble size `ensemblesize`, and a maximum number of clusters to consider (`maxK`).  The function should run `ensemble_run` for all values of `K` from 1 to `maxK`, and return a numpy array of length `maxK` with the best case cost for each value of `K`. 

A random seed is passed in to the function for the autograder. Please pass this seed on to the `ensemble_run` function.

In [ ]:
def K_iteration(D, ensemblesize, maxK,random_seed=None):
    if random_seed is not None:
        np.random.seed(seed=random_seed)
    best_cost = np.empty(maxK)
    for ind in range(maxK):
        ...   # TODO
    return best_cost

In [ ]:
ensemblesize=100
maxK=10
best_cost = K_iteration(D, ensemblesize, maxK,random_seed=452)

In [ ]:
grader.check("q10")

# 11) Selecting K

Running the cell below will produce a plot showing the performance of the algorithm for different values of `K` ranging from 1 to 10 (calculated in the previous part). The top row shows the cost function, which naturally decreases monotonically with increasing `K`. The bottom plot shows the percentage improvement from one value of `K` to the next. What is the best value of `K` to choose according to these plots? 

In [ ]:
ensemblesize=100
maxK=10
best_cost = K_iteration(D, ensemblesize, maxK,random_seed=452)

fig, ax = plt.subplots(figsize=(8,5),nrows=2,sharex=True)

ax[0].plot(range(1,maxK+1),best_cost,
         linewidth=3,
         marker='o',
         markersize=8)
ax[0].grid()
ax[0].set_ylabel('Cost')

ax[1].plot(range(2,maxK+1),100*np.abs(np.diff(best_cost)) / best_cost[1:],
         linewidth=3,
         marker='o',
         markersize=8)
ax[1].grid()
ax[1].set_ylabel('Cost improvement')
ax[1].set_xlabel('K')

In [ ]:
Kideal = ...   # TODO

In [ ]:
grader.check("q11")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Make sure you submit the .zip file to Gradescope.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)